# IA: MongoDB Aggregation Pipelines

Answer the following questions using the `movies` collection of the `sample_mflix` database.

You will add your query to the document here and will also submit your exported results for each question.

**NOTE:** You will build your pipelines using the visual editor in either MongoDB Compass or the MongoDB web interface and copy/paste the queries here. You **WILL NOT** try to run the queries themselves from this document!! This is just a convenient format to store and submit your queries for grading.

## 1: Counting Cast Members
### 3 points

Create a dataset of movies (`type`) released in 1996 (`year`) whose `imdb.rating` is greater than 8. The documents in the dataset should contain four fields:
1. `title`
2. `year`
3. `imdb.rating`
4. **NUMBER** of cast members listed in `cast`

Paste your query below.

Export your dataset AS A CSV FILE named `1996_cast.csv`

In [ ]:
db.getCollection('movies').aggregate(
  [
    {
      $project: {
        _id: 0,
        type: 1,
        year: 1,
        rating: '$imdb.rating',
        title: 1,
        cast: {
          $size: { $ifNull: ['$cast', []] }
        }
      }
    },
    {
      $match: {
        type: 'movie',
        year: 1996,
        rating: { $gt: 8 }
      }
    },
    {
      $project: {
        title: 1,
        year: 1,
        imdb_rating: '$rating',
        cast_member: '$cast'
      }
    }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

## 2: Summarizing by Plot

### 4 points

Create a dataset of movies (`type`) whose plot (`plot`) specifically mentions "Italy" or "Italian" (any case). Only include movies that were released in the "USA" (`countries`) and have more than one language (`languages`). Sort the dataset alphabetically by year of release (`year`) and title (`title`). Include all fields.

Note: if a movie was released in the USA as well as other countries, it should still appear in the dataset.

Export your dataset AS A JSON FILE named `italy_movies.json`

In [ ]:
db.getCollection('movies').aggregate(
  [
    {
      $match: {
        type: 'movie',
        plot: {
          $regex: '(italy|italian)',
          $options: 'i'
        },
        countries: { $in: ['USA'] },
        $expr: {
          $gt: [
            {
              $size: {
                $ifNull: ['$languages', []]
              }
            },
            1
          ]
        }
      }
    },
    { $sort: { year: 1, title: 1 } }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

## 3: Grouping by Genre

### 5 points

Create a dataset of genres (`genres`) for movies (`type`) released from 1950-1959 (inclusive) (`year`). Each document in the dataset should represent ONE genre and should include a collection of movie titles (`title`) that appear in that genre.

The documents in the dataset should contain two fields:
1. `genre`
2. `movies` (an array of movie `title`s)

Movies that have more than one genre should appear in each genre's document.

Export your dataset AS A JSON FILE named `1950s_movies.json`.

In [ ]:
db.getCollection('movies').aggregate(
  [
    {
      $project: {
        _id: 0,
        genres: 1,
        type: 1,
        year: 1,
        title: 1
      }
    },
    {
      $match: {
        type: 'movie',
        year: { $gte: 1950, $lte: 1959 }
      }
    },
    { $unwind: { path: '$genres' } },
    {
      $group: {
        _id: { genre: '$genres' },
        movies: { $push: '$title' }
      }
    },
    {
      $project: {
        _id: 0,
        genre: '$_id.genre',
        movies: 1
      }
    }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

## 4: Cast Member Movie Ratings

### 6 points

Create a dataset of cast members (`cast`) for all movies (`type`) in the database. Each document in the dataset should represent ONE cast member and should include the AVERAGE IMDB rating (`imdb.rating`) for all movies in which they have appeared. Sort the dataset alphabetically by cast member.

The documents in the dataset should contain two fields:
1. `cast_member`
2. `average_rating`

Export your dataset AS A CSV FILE named `cast_ratings.csv`

In [ ]:
db.getCollection('movies').aggregate(
  [
    {
      $project: {
        _id: 0,
        cast: 1,
        type: 1,
        rating: '$imdb.rating'
      }
    },
    { $match: { type: 'movie' } },
    { $unwind: { path: '$cast' } },
    {
      $group: {
        _id: { cast_member: '$cast' },
        average_rating: { $avg: '$rating' }
      }
    },
    {
      $project: {
        _id: 0,
        cast_member: '$_id.cast_member',
        average_rating: {
          $round: ['$average_rating', 1]
        }
      }
    },
    { $sort: { cast_member: 1 } }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

## 5: Director Activity By Year

### 7 points

Create a dataset of movies (`type`) for each year (`year`) in the database. Each document should represent ONE year. 

The document should contain an array field named `directors` that contains an object for each distinct director (`director`) who directed at least one movie in that year. The object should also note the number of movies that director directed for that year. Only include directors who directed more than 1 movie in any given year.

### Example (demonstrating structure only)
```
{
    "year" : "2000",
    "directors" : [
        { 
            "director" : "John Smith",
            "movie_count" : 3 
        },
        { 
            "director" : "Mary Edwards",
            "movie_count" : 2 
        },
    ]
}
```

Export your dataset AS A JSON FILE named `years_directors.json`

In [ ]:
db.getCollection('movies').aggregate(
  [
    {
      $project: {
        year: 1,
        directors: 1,
        title: 1,
        type: 1
      }
    },
    { $match: { type: 'movie' } },
    { $unwind: { path: '$directors' } },
    {
      $group: {
        _id: {
          year: '$year',
          director: '$directors'
        },
        movie_count: { $sum: 1 }
      }
    },
    { $match: { movie_count: { $gt: 1 } } },
    {
      $group: {
        _id: { year: '$_id.year' },
        directors: {
          $push: {
            director: '$_id.director',
            movie_count: '$movie_count'
          }
        }
      }
    },
    {
      $project: {
        _id: 0,
        year: '$_id.year',
        director: '$directors'
      }
    }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);